# Real VS Fake Faces Classification

## Model evaluation

In [1]:
import os
import pandas as pd

import utils
import model

## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** path to the config.yml file used for trainning.
- **WEIGHTS:** path to the model weights (.h5 file) to evaluate


In [2]:
CONFIG_YML = "config.yml"
WEIGHTS = "weights/model.better.h5"

In [3]:
config = utils.load_config(CONFIG_YML)
config

{'seed': 21,
 'data': {'directory': 'data/subset',
  'directory_bis': '../data/subset',
  'test_dataset': '../data/rd_test_dataset',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [256, 256],
  'batch_size': 20},
 'model': {'weights': 'imagenet',
  'input_shape': [256, 256, 3],
  'classes': 2,
  'data_aug_layer': None,
  'regularizer': 'l2',
  'trainable': True},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.005,
    'epsilon': 0.05,
    'amsgrad': True}},
  'loss': 'binary_crossentropy',
  'metrics': ['categorical_accuracy']},
 'fit': {'epochs': 30,
  'callbacks': {'model_checkpoint': {'filepath': 'model/weights/model.better.h5',
    'save_best_only': True},
   'early_stopping': {'monitor': 'val_loss',
    'patience': 2,
    'restore_best_weights': True},
   'tensor_board': {'log_dir': 'model/weights/logs'}}}}

In [4]:
# CLASSES = utils.get_class_names(config)
CLASSES = sorted(os.listdir(os.path.join(config["data"]["directory_bis"])))
TEST_FOLDER = config['data']['test_dataset']

In [5]:
cnn_model = model.create_model(weights=WEIGHTS)

print(cnn_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 256, 256, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 256, 256, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                             

## Get predictions from testing dataset

In [6]:
filename, predictions = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=CLASSES,
)

1/1 [==============================] - 0s 121ms/step


If all the images in the test dataset are fake, we then obtain:

In [17]:
test_df = pd.DataFrame({'filename': filename, 'predictions': predictions})
test_df.to_csv('test_df.csv')

COUNTS = test_df['predictions'].value_counts()
ACCURACY = COUNTS['fake'] / len(test_df)

In [23]:
print("Accuracy of the model on this dataset is", round(ACCURACY, 2)*100, "%.")

Accuracy of the model on this dataset is 91.0 %.
